In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/"
predictions = pd.read_csv(path + 'mono_labels.csv')

In [ ]:
import pickle 
# dc = pickle.load(open(,'r'))
# with open(path + 'labels_kmeans_and_crf.pkl', 'rb') as pickle_file:
#     content = pickle.load(pickle_file)

predictions = content['crf']

In [ ]:
content['crf']

['B',
 'B',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'L',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'L',
 'L',
 'L'

In [ ]:
predictions = predictions[predictions.columns[1]].to_numpy()

AttributeError: ignored

In [ ]:
def convert(ip):
  #expects a list or numpy list of states 'L' or 'B' or 'C' for low nfr, background and center respt
  # returns a list of 'P' = peaks 'B' = background 
  lefts = []
  rights = []
  flag = 0
  for i in range(len(ip)):
    if (ip[i] == 'L'):
      lefts.append(flag)
    elif (ip[i] == 'B'):
      flag = 1
      lefts.append(flag)
    else:
      flag = 2
      lefts.append(flag)

  for i in reversed(ip):
    if (i == 'B'):
      flag = 1
    elif (i == 'C'):
      flag = 2
    rights.append(flag)

  rights = [i for i in reversed(rights)]

  ret = []
  for i in range(len(ip)):
    if (lefts[i] == 2 or rights[i] == 2):
      ret.append(1)
    else:
      ret.append(0)
  return ret

In [ ]:
final_predictions = convert(predictions)

In [ ]:
sum(final_predictions)

425592

In [ ]:
path += 'project/'
TN = 0
FP = 0
real_negatives = open(path + 'real_negative.bed','r').read()
lines = real_negatives.split('\n')
print(len(lines))
RN = []
for i in lines:
  j = i.split('\t')
  if(len(j) == 1):
    continue
  # print(j)
  if(len(j) < 3):
    print(j)
  RN.append((int(j[1]) - 29000000,int(j[2]) - 29000000))

real_positives = open(path + 'real_positive.bed','r').read()
lines = real_positives.split('\n')
print(len(lines))
RP = []
for i in lines:
  j = i.split('\t')
  if(len(j) == 1):
    continue
  # print(j)
  
  if(len(j) < 3):
    print(j)
  RP.append((int(j[1]) - 29000000,int(j[2]) - 29000000))

FileNotFoundError: ignored

In [ ]:

max_positions = len(final_predictions) - 1
def total_positions(peak):
  return peak[1] - peak[0] + 1 


data = {}
chr = 'chr22'
count = 0

labels = [-1]*(max_positions+1)

for i in RN:
  labels[i[0]:i[1] + 1] = [0]*total_positions(i)
for i in RP:
  for j in range(i[1] - i[0] + 1):
    if (j + i[0] <= max_positions and labels[j + i[0]] == 0):
      labels[j+i[0]] = -1
      count += 1
    else:
      if (j + i[0] <= max_positions):
        labels[j+i[0]] = 1
labels = np.array(labels[:max_positions + 1])
preds = np.array(final_predictions)

preds = preds[labels != -1]
labels = labels[labels != -1]

TP = (preds[labels == 1] == 1).sum()
FN = (preds[labels == 1] == 0).sum()
TN = (preds[labels == 0] == 0).sum()
FP = (preds[labels == 0] == 1).sum()
data[chr] = {'TP':TP,'TN':TN,'FP':FP,'FN':FN}

In [ ]:
total_tp = 0
total_fp = 0 
total_fn = 0
total_tn = 0

total_tp += data[chr]['TP']
total_fp += data[chr]['FP']
total_fn += data[chr]['FN']
total_tn += data[chr]['TN']

precision = total_tp / (total_tp + total_fp)
recall = total_tp / (total_tp + total_fn)
print('precision = ',precision)
print('recall = ',recall)

precision =  0.5693024960299686
recall =  0.6115071728481456


In [ ]:
peaks = []

In [ ]:
start = 0
end =  0
prev = 0
for i,j in enumerate(final_predictions):
  # print(i,j)
  if j == 1:
    if prev == 0:
      start = i + 29000000
      end = start
    else:
      end = end + 1
  elif prev == 1:
    peaks.append((start,end))
  prev =  j

if prev == 1:
  peaks.append((start,end))

      

In [ ]:
peaks

[(29001323, 29001521),
 (29002268, 29002311),
 (29002457, 29002862),
 (29006252, 29007133),
 (29008033, 29008566),
 (29009801, 29009844),
 (29011566, 29011891),
 (29012856, 29013621),
 (29013692, 29014941),
 (29015180, 29015237),
 (29016117, 29017791),
 (29018561, 29019922),
 (29022699, 29022743),
 (29023296, 29024079),
 (29024443, 29024507),
 (29025093, 29025855),
 (29025897, 29027131),
 (29028164, 29028612),
 (29028638, 29029180),
 (29031360, 29032317),
 (29032589, 29033184),
 (29033520, 29034387),
 (29037895, 29038784),
 (29039366, 29039731),
 (29040408, 29041333),
 (29041421, 29041847),
 (29042355, 29042760),
 (29042856, 29043279),
 (29043572, 29044513),
 (29044597, 29045323),
 (29047923, 29048641),
 (29048646, 29049579),
 (29049738, 29050202),
 (29051337, 29051598),
 (29051856, 29052131),
 (29052249, 29052681),
 (29053038, 29053082),
 (29055005, 29055054),
 (29055320, 29055605),
 (29056396, 29057197),
 (29058316, 29058663),
 (29058717, 29059947),
 (29060452, 29061099),
 (29061401,

In [ ]:
with open("out_crf.bed", "w") as f:
  for i,j in peaks:
    f.write(F"chr22\t{i}\t{j}\n")

In [ ]:
from google.colab import files
files.download('out_crf.bed')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>